### Task
Write a class that takes a set of 3d points and estimate the camera pose

In [1]:
import numpy as np
import cv2

class CameraPoseEstimator:
    def __init__(self, K: np.ndarray, dist_coefs: np.ndarray = None):
        """
        Initialize the estimator with camera intrinsics.

        Parameters:
        - K: (3x3) camera intrinsic matrix
        - dist_coefs: distortion coefficients (default: None for no distortion)
        """
        self.K = K
        self.dist_coefs = dist_coefs

    def estimate_pose(self, points_3d: np.ndarray, points_2d: np.ndarray, use_ransac=True):
        """
        Estimate the camera pose (R, t) from 3D-2D correspondences.

        Parameters:
        - points_3d: Nx3 array of 3D world coordinates
        - points_2d: Nx2 array of 2D image coordinates
        - use_ransac: if True, uses RANSAC for robustness

        Returns:
        - R: 3x3 rotation matrix
        - t: 3x1 translation vector
        - inliers: list of inlier indices (None if RANSAC not used)
        """
        assert points_3d.shape[0] >= 4, "At least 4 points are required for pose estimation"

        if use_ransac:
            success, rvec, tvec, inliers = cv2.solvePnPRansac(
                objectPoints=points_3d,
                imagePoints=points_2d,
                cameraMatrix=self.K,
                distCoeffs=self.dist_coefs,
                flags=cv2.SOLVEPNP_ITERATIVE
            )
        else:
            success, rvec, tvec = cv2.solvePnP(
                objectPoints=points_3d,
                imagePoints=points_2d,
                cameraMatrix=self.K,
                distCoeffs=self.dist_coefs,
                flags=cv2.SOLVEPNP_ITERATIVE
            )
            inliers = None

        if not success:
            raise RuntimeError("Pose estimation failed.")

        R, _ = cv2.Rodrigues(rvec)  # Convert from rotation vector to matrix
        t = tvec.reshape(3, 1)

        return R, t, inliers


In [ ]:
# Assuming we have known 3D points and their 2D projections

# Camera intrinsics
K = np.array([[800, 0, 320],
              [0, 800, 240],
              [0,   0,   1]])

# Some known 3D points (in world frame)
points_3d = np.array([
    [0, 0, 5],
    [1, 0, 5],
    [0, 1, 5],
    [1, 1, 5],
    [0.5, 0.5, 6]
], dtype=np.float32)

# Their 2D projections (in image plane)
points_2d = np.array([
    [320, 240],
    [400, 240],
    [320, 320],
    [400, 320],
    [360, 280]
], dtype=np.float32)

# Estimate the pose
estimator = CameraPoseEstimator(K)
R, t, inliers = estimator.estimate_pose(points_3d, points_2d)

print("Estimated Rotation Matrix:\n", R)
print("Estimated Translation Vector:\n", t)
print("Inlier indices:\n", inliers)


Estimated Rotation Matrix:
 [[ 0.99867529 -0.00132471  0.0514385 ]
 [-0.00132471  0.99867529  0.0514385 ]
 [-0.0514385  -0.0514385   0.99735057]]
Estimated Translation Vector:
 [[-0.2558438 ]
 [-0.2558438 ]
 [ 5.09252611]]
Inlier indices:
 [[0]
 [1]
 [2]
 [3]
 [4]]
